# HW 5: Mixed-Integer Programming
---
## OR/ISE 501 - Fall 2021
__Assigned:__ Tue, 12 Oct (Groups of 2)  
__Due:__ 11:59p, Tue, 19 Oct

__Group Members:__ Kamlesh Pandey & Ronak Dhongde

Please use the Code cells in this Jupyter notebook to answer each of the following questions. You can add additional cells for each question if that helps in organizing your solution. Please run all of the cells in your notebook and then submit it via Moodle. (There is a _Run All Cells_ command under the _Run_ menu.)

---
__(1)__ Continuing with Question 2 from _HW 3_, just before purchasing the items, you found out that, in addition to the 30-pound weight and three-cubic-foot cube limits, the items can only be purchased in unit-sized packages. The table below lists the weight (lb) and cube (in3) of each unit along with maximum and minimum order size (units) and the sale and purchase prices (\$/unit). Either zero units or at least the minimum order size of each item must be purchased. Also, due to cross-contamination of odors, ginger and tea cannot both be purchased.

```
     │ Item     UnitWt  UnitCube  MaxOrder  MinOrder  SalePrice  PurPrice
─────┼────────────────────────────────────────────────────────────────────
   1 │ Coffee        4       216         5         1         24         8
   2 │ Peppers       3       648         2         1         45        18
   3 │ Ginger        1        58         5         2          8         1
   4 │ Tea           1       864         6         2         35         5
   5 │ Candy         2       173         4         3         12         2
```

__(1*a*)__ How many units of each item should be purchased?

In [1]:
using Plots,JuMP, GLPK, DataFrames

df = DataFrame(item = ["Coffee", "Peppers", "Ginger", "Tea", "Candy"],
                UnitWt = [4,3,1,1,2],
                UnitCube = [216*(0.08)^3, 648*(0.08)^3, 58*(0.08)^3, 864*(0.08)^3, 173*(0.08)^3],   #convert into foot
                MaxOrder = [5,2,5,6,4],  
                MinOrder = [1,1,2,2,3], 
                SalePrice = [24,45,8,35,12],
                PurPrice = [8,18,1,5,2])

,item,UnitWt,UnitCube,MaxOrder,MinOrder,SalePrice,PurPrice
,String,Int64,Float64,Int64,Int64,Int64,Int64
1,Coffee,4,0.110592,5,1,24,8
2,Peppers,3,0.331776,2,1,45,18
3,Ginger,1,0.029696,5,2,8,1
4,Tea,1,0.442368,6,2,35,5
5,Candy,2,0.088576,4,3,12,2


In [2]:
model = Model(GLPK.Optimizer)
u , p ,c, v = df.MaxOrder, df.PurPrice, df.SalePrice, df.UnitCube
min_Order = df.MinOrder
Unit_Weight = df.UnitWt

N = 1:length(p)
@variable(model, 0<= x[i=N] <= u[i], Int) #integer variable 
@variable(model, y[i = N], Bin)          # Binary variable
@objective(model, Max, sum((c[i] - p[i])*x[i] for i in N)) 
@constraint(model, sum(p[i]*x[i] for i in N) <= 120)           #Budget constraint
@constraint(model, sum(x[i]*Unit_Weight[i] for i in N) <=30)   #Weigh constraint
@constraint(model, sum(v[i]*x[i] for i in N) <= 3)             #volume constraint

for i in N                                           #min order constraint
    @constraint(model, x[i] >= y[i]*min_Order[i])
    @constraint(model, x[i]<=y[i]*u[i])
end

@constraint(model, y[3]+y[4] == 1)          # tea and ginger constraint

print(model)

In [ ]:
optimize!(model)
x° = value.(x)
N1° = objective_value(model)
println("Total profit based on the given constraints: ", N1°)

df_output1 = DataFrame( Item_Name = ["Coffee", "Peppers", "Ginger", "Tea", "Candy"], Optimum_Purchase_Qty_lb = [x°[1], x°[2], x°[3], x°[4], x°[5]])
df_output1

Total profit based on the given constraints: 254.0


__(1*b*)__ What is the impact on profit of the ginger and tea restriction?

Ginger and Tea restriction causing decrease in total profit, if we remove ginger/tea constraint there is an increase of 4.7% in total profit.

In [ ]:
model2 = Model(GLPK.Optimizer)
u , p ,c, v = df.MaxOrder, df.PurPrice, df.SalePrice, df.UnitCube
min_Order = df.MinOrder
Unit_Weight = df.UnitWt

N = 1:length(p)
@variable(model2, 0<= x[i=N] <= u[i], Int) #integer variable 
@variable(model2, y[i = N], Bin)          # Binary variable
@objective(model2, Max, sum((c[i] - p[i])*x[i] for i in N)) 
@constraint(model2, sum(p[i]*x[i] for i in N) <= 120)           #Budget constraint
@constraint(model2, sum(x[i]*Unit_Weight[i] for i in N) <=30)   #Weigh constraint
@constraint(model2, sum(v[i]*x[i] for i in N) <= 3)             #volume constraint

for i in N                                           #min order constraint
    @constraint(model2, x[i] >= y[i]*min_Order[i])
    @constraint(model2, x[i]<=y[i]*u[i])
end


# Removing tea and ginger constraint from the model 

optimize!(model2)
x° = value.(x)
N2° = objective_value(model2)
println("Total profit by removing tea and ginger constraint: ", N2°)

df_output2 = DataFrame( Item_Name = ["Coffee", "Peppers", "Ginger", "Tea", "Candy"], Optimum_Purchase_Qty_lb = [x°[1], x°[2], x°[3], x°[4], x°[5]])
df_output2


In [ ]:
println("The impact on profit of the ginger and tea restriction is \$ ", (N2° - N1°))

---
__(2)__ Weekend staffing needs to be determined at a clinic, and six employees are available. In order to operate, the clinic must have at least one staff member working that is certified in each of twelve different skills. Employee A is certified in skills 1, 4, and 11; employee B in 1, 3, 6, 7, 8, and 10; C in 2, 5, 8, 9, 10, 11, and 12; D in 6, 7, 10, and 11; E in 2, 4, 5, 8, 9, and 12; and employee F in skills 3, 4, 7, 8, 10, and 12. Each employee selected works for the entire time the clinic is open on the weekend, and the fully burdened cost of employees A to F is \\$25, 70, 40, 30, 50, and 40 per hour, respectively. Determine which employees should staff the clinic.

In [ ]:
M = 1:12
N = 1:6
Mi = [[1,4,11],[1,3,6,7,8,10],[2,5,8,9,10,11,12],[6,7,10,11],[2,4,5,8,9,12],[3,4,7,8,10,12]]
A = zeros(length(M),length(N))     # A = objects x subsets
for i = 1:size(A,2)
    A[Mi[i],i] .= 1
end
A

In [ ]:
using JuMP, Cbc
model = Model(Cbc.Optimizer)
wages = [25,70,40,30,50,40]
@variable(model, x[1:length(N)], Bin)
@objective(model, Min, sum(x[i]*wages[i] for i in N))
@constraint(model, [j in M], sum(A[j,i]*x[i] for i in N) >=1)
set_optimizer_attribute(model, "loglevel", 1)
optimize!(model)
TC1, x1 = objective_value(model), value.(x)
salary = sum((value.(x)).*wages)
salary, TC1, x1

---
__(3)__ There are 100 cities located throughout a region of a state, and public service radio transmitters can be located in any of the cities. The signal from a transmitter can only reach other cities that are within 30 units of distance because of signal attenuation. The coordinates of the cities can be generated by running the following code:

```
using Random, Statistics
Random.seed!(62738462)
P = 200*rand(100,2)
```

__(3*a*)__ Determine the cities where transmitters should be located. 

In [ ]:
using Random, Statistics
Random.seed!(62738462)
P = 200*rand(100,2)
dist(x,P) = sqrt.(sum((x' .- P).^2, dims = 2))
D = vcat([dist(P[i,:],P) for i in 1:size(P,1)]'...)

In [ ]:
A = D .< 30

In [ ]:
using JuMP, Cbc
function setcover(A)
    M = 1:size(A,1)
    N = 1:size(A,2)
    model = Model(Cbc.Optimizer)
    @variable(model, x[1:length(N)], Bin)
    @objective(model, Min, sum(x[i] for i in N))
    @constraint(model, [j in M], sum(A[j,i]*x[i] for i in N) >= 1)
    set_optimizer_attribute(model, "loglevel", 0)
    set_optimizer_attribute(model, "seconds", 60.00)
    optimize!(model)
    println(solution_summary(model).termination_status)
    return findall(value.(x) .> 0)
end
Z = setcover(A)

In [ ]:
println("The location of the first transmitter should be in city: ", Z[1])
println("Followed by: ")
for j in (2:(length(Z)-1))
    println("The location of the next transmitter should be in city: ", Z[j])
    println("Followed by: ")
end
println("The location of the last transmitter should be in city: ", Z[end])

__(3*b*)__ Due to interference, the signals from transmitters that are within 60 units of distance from each other should use different frequencies. Determine the minimum number of different frequencies needed for the transmitters. 

In [ ]:
A = D .< 60      #setting dist to 60

In [ ]:
Z = setcover(A)

In [ ]:
println("The location of the first transmitter should be in city: ", Z[1])
println("Followed by: ")
for j in (2:(length(Z)-1))
    println("The location of the next transmitter should be in city: ", Z[j])
    println("Followed by: ")
end
println("The location of the last transmitter should be in city: ", Z[end])

---
__(4)__ A single product is produced in a single-stage production process. A 13-week rolling horizon is used for planning production. The process has a capacity of 60 tons per week. The forecasted demand (ton) over the horizon is provided in *HW5-Data-forecasted_demand.csv*. Production data for the past year is provided in *HW5-Data-historical_prod_cost.csv*, where production (ton/wk) and cost (\\$000/wk) are listed for the weeks in which the process was operating. The inventory carrying rate is estimated as 0.125 per week. Currently, there are 13 tons of finished product in storage, and this same amount should be in inventory at the end of the planning period. There is enough space to store up to 120 tons of finished product. Determine the best production plan over the planning horizon.

In [ ]:
using DataFrames, CSV
df1 = DataFrame(CSV.File("HW5-Data-historical_prod_cost.csv"))
df2 = DataFrame(CSV.File("HW5-Data-forecasted_demand.csv"))
df1, df2

In [ ]:
u = [60]          # capacity of each stage 
d = df2.Demand    # demand in each period 
cp = df1.Cost     # Production cost for each stage ($)
h = 0.125         # monthly inventory carrying rate (1/month)
ci = cp*h/u       # inventory cost for each stage ($)
y0 = 13           # initial inventory 
yt = 13           # final inventory
m = length(u)     # number of production stages
t = length(d)     # number of periods of production
ylb = [y0 zeros(m,t-1) yt]       # same LB and UB to fix inital and final inventory levels
yub = [y0 repeat([120],m,t-1) yt] # max inventory storage level

In [ ]:
using JuMP, Cbc
model = Model(Cbc.Optimizer)
@variable(model, 0 <= X[i=1:m, j=1:t])
@variable(model, ylb[i,j] <= y[i=1:m, j=1:t+1] <= yub[i,j])
@variable(model, Z[i=1:m, j=1:t], Bin)
@objective(model, Min, sum(cp[i]*X[i,j] for i=1:m, j=1:t ) + 
    sum(ci[i]*y[i,j] for i=1:m, j=2:t+1 )  ) 
for j = 1:t
    for i = 1:m-1                                      # (a) flow-balance all but last stage
        @constraint(model, -X[i,j] + X[i+1,j] - y[i,j] + y[i,j+1] == 0 )   
    end                                                # (b) flow-balance for last stage
    @constraint(model, -X[m,j] - y[m,j] + y[m,j+1] == -d[j] )              
    for i = 1:m                                        # (c) production capacity
        @constraint(model, X[i,j] <= u[i]*Z[i,j] )                         
    end
end
set_optimizer_attribute(model, "logLevel", 0)
set_optimizer_attribute(model, "seconds", 30.0)
optimize!(model)
println(solution_summary(model).termination_status)
TCᵒ, Xᵒ, Yᵒ, Zᵒ = objective_value(model), value.(X), value.(y), round.(value.(Z))

In [ ]:
Matrix([Xᵒ; d'])

In [ ]:
Matrix(Yᵒ)

In [ ]:
Matrix(Zᵒ)

---
__(5)__ A plant can use a three-stage process to produce three products. A 13-week rolling horizon is used for planning production. The products’ forecasted demand (ton) over the horizon is provided in *HW5-Data-forecasted_demand_2.csv*.The plant can produce up to 600, 550, and 500; 500, 450, and 350; and 400, 350, and 300 tons per week for each stage of each product, respectively; has production costs of \\$12, 75, and 35; 20, 130, and 60; and 16, 100, and 50 per ton for each stage of each product, respectively; and has costs of \$4000, 900, and 500; 6000, 1100, and 600; and 5000, 1000, and 600 per set up for each stage of each product, respectively. The annual inventory carrying rate is 0.3. The plant is currently set up to produce each stage of the first product, and the first three weeks of demand for the other two products are being held as inventory at each stage. The final inventory for all stages of all products is zero. Determine the amount of each product that should be produced in order to for the total costs over the planning horizon to be within 1% of the optimal value.  

In [ ]:
using DataFrames, CSV
D = Int.(Matrix(DataFrame(CSV.File("HW5-Data-forecasted_demand_2.csv", header=false))))
D

In [ ]:
inv3 = (D[:,3])[1] + (D[:,3])[2] + (D[:,3])[3]

In [ ]:
inv2 = (D[:,2])[1] + (D[:,2])[2] + (D[:,2])[3]

In [ ]:
T = size(D,1)    #Number of periods
U = [600 550 500; 500 450 350; 400 350 300]      #capacity at each stage of each product
m = size(U,1)     #No. of prod. stages
n = size(U,2)     #No. of products
Cp = [12 75 35; 20 130 60; 16 100 50]      #Prod cost for each stage
h = (0.3)/(365.25/7)                      #Weekly inventory carrying rate
Ci = cumsum(Cp, dims = 1)*h                #inv cost for each stage/product
Cs = [4000 900 500; 6000 1100 600; 5000 1000 600]   #setup cost for each stage/product
Y0 = [0 inv2 inv3; 0 inv2 inv3; 0 inv2 inv3]       #initial inventory for each prod

In [ ]:
Yf =zeros(m,n)    #final level of inventory
Z0 = [1 0 0; 1 0 0; 1 0 0];      #initial setup at each stage/product

Ylb = [reshape(Y0,m,1,n) zeros(m,T-1,n) reshape(Yf,m,1,n)]
Yub =  [reshape(Y0,m,1,n) repeat([Inf],m,T-1,n) reshape(Yf,m,1,n)]

In [ ]:
using JuMP, Cbc
model = Model(Cbc.Optimizer)
@variable(model, 0 <= X[i=1:m, j=1:T, k=1:n] )
@variable(model, Ylb[i,j,k] <= Y[i=1:m, j=1:T+1, k=1:n] <= Yub[i,j,k] )
@variable(model, Z[i=1:m, j=1:T, k=1:n], Bin )
@variable(model, V[i=1:m, j=1:T, k=1:n], Bin )
@objective(model, Min, 
    sum(Cp[i,k]*X[i,j,k] for i=1:m, j=1:T, k=1:n ) + 
    sum(Ci[i,k]*Y[i,j,k] for i=1:m, j=2:T+1, k=1:n ) + 
    sum(Cs[i,k]*V[i,j,k] for i=1:m, j=1:T, k=1:n ) )
for k = 1:n
    for j = 1:T
        for i = 1:m-1                                  # (a) flow-balance all but last stage
            @constraint(model, -X[i,j,k] + X[i+1,j,k] - Y[i,j,k] + Y[i,j+1,k] == 0 )    
        end                                            # (b) flow-balance for last stage
        @constraint(model, -X[m,j,k] - Y[m,j,k] + Y[m,j+1,k] == -D[j,k] )               
        for i = 1:m                                    # (c) production capacity
            @constraint(model, X[i,j,k] <= U[i,k]*Z[i,j,k] )
        end
    end
    for i = 1:m                                         # (d) setup for first stage
        @constraint(model, -V[i,1,k] + Z[i,1,k] <= Z0[i,k] )
        for j = 2:T                                     # (e) setup all but first stage
            @constraint(model, -V[i,j,k] + Z[i,j,k] - Z[i,j-1,k] <= 0 )
        end
    end
end
for i = 1:m
    for j = 1:T                                         # (f) linking
        @constraint(model, sum(Z[i,j,k] for k=1:n) == 1 )              
    end
end
set_optimizer_attribute(model, "logLevel", 0)
set_optimizer_attribute(model, "seconds", 30.0)
optimize!(model)
println(solution_summary(model).termination_status)
TCᵒ,Xᵒ,Yᵒ,Zᵒ,Vᵒ = 
objective_value(model), value.(X), value.(Y), round.(value.(Z)), round.(value.(V))

In [ ]:
Matrix([Xᵒ[:,:,1]; D[:,1]'])   # Product 1 production level/Demand

In [ ]:
Matrix([Xᵒ[:,:,2]; D[:,2]'])   # Product 2 production level/Demand

In [ ]:
Matrix([Xᵒ[:,:,3]; D[:,3]'])   # Product 3 production level/Demand

In [ ]:
Matrix(Yᵒ[:,:,1])              # Product 1 inventory level

In [ ]:
Matrix(Yᵒ[:,:,2])              # Product 2 inventory level

In [ ]:
Matrix(Yᵒ[:,:,3])              # Product 3 inventory level

In [ ]:
Matrix(Zᵒ[:,:,1])              # Product 1 production indicator

In [ ]:
Matrix(Zᵒ[:,:,2])              # Product 2 production indicator

In [ ]:
Matrix(Zᵒ[:,:,3])              # Product 3 production indicator

In [ ]:
Matrix(Vᵒ[:,:,1])              # Product 1 setup indicator

In [ ]:
Matrix(Vᵒ[:,:,2])              # Product 2 setup indicator

In [ ]:
Matrix(Vᵒ[:,:,3])              # Product 3 setup indicator

In [ ]:
size(Cp),size(Xᵒ)   # Cp not compatible size with Xᵒ, need to reshape to (3, 1, 2)

In [ ]:
TCp = sum((reshape(Cp,m,1,n).*Xᵒ)[:])   # Reshaped Cp broadcast for multiplication with Xᵒ
TCi = sum((reshape(Ci,m,1,n).*Yᵒ[:,2:end,:])[:])   # First period excluded
TCs = sum((reshape(Cs,m,1,n).*Vᵒ)[:])
TCp, TCi, TCs

In [ ]:
TCp+TCi+TCs, TCᵒ